## Data Preparation

Step 1. Load the data.

In [29]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import kagglehub
import findspark

import pyspark
from pyspark.sql import SparkSession


c:\Users\Jean\anaconda3\envs\Dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load in the data using pyspark
# findspark.init()
# spark=SparkSession.builder.appName("Local CSV").getOrCreate()

# fraud_test_df = spark.read.csv('C:\Users\Jean\Documents\Bootcamp\Project_4\fraudTest.csv', header=True, inferSchema=True)
# fraud_test_df = spark.read.csv('C:/Users/Jean/Documents/Bootcamp/Project_4/fraudTest.csv')

# fraud_train_df = spark.read.csv('C:\Users\Jean\Documents\Bootcamp\Project_4\fraudTrain.csv', header=True, inferSchema=True)
# fraud_train_df = spark.read.csv('C:/Users/Jean/Documents/Bootcamp/Project_4/fraudTrain.csv')


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [30]:
# Define path to Kagglehub dataset

path = kagglehub.dataset_download("kartik2112/fraud-detection")

In [31]:
fraud_test_csv = r"C:\Users\Jean\.cache\kagglehub\datasets\kartik2112\fraud-detection\versions\1\fraudTest.csv"
fraud_train_csv = r"C:\Users\Jean\.cache\kagglehub\datasets\kartik2112\fraud-detection\versions\1\fraudTrain.csv"

In [35]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
fraud_test_df_complete = pd.read_csv(fraud_test_csv)
fraud_train_df_complete = pd.read_csv(fraud_train_csv)


In [ ]:
# Check the fraud_test_df_complete information

fraud_test_df_complete.info()
fraud_test_df_complete.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  int64  
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755302,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


In [ ]:
# Check the fraud_train_df_complete information

fraud_train_df_complete.info()
fraud_train_df_complete.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [40]:
# Drop unwanted columns
fraud_test_df = fraud_test_df_complete[["trans_date_trans_time", "merchant", "amt", "zip", "merch_lat", "merch_long", "is_fraud"]]

fraud_train_df = fraud_train_df_complete[["trans_date_trans_time", "merchant", "amt", "zip", "merch_lat", "merch_long", "is_fraud"]]

In [42]:
# Check the fraud_test_df information

fraud_test_df.info()
fraud_test_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  555719 non-null  object 
 1   merchant               555719 non-null  object 
 2   amt                    555719 non-null  float64
 3   zip                    555719 non-null  int64  
 4   merch_lat              555719 non-null  float64
 5   merch_long             555719 non-null  float64
 6   is_fraud               555719 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 29.7+ MB


,amt,zip,merch_lat,merch_long,is_fraud
count,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000
mean,69.392810,48842.628015,38.542798,-90.231380,0.003860
std,156.745941,26855.283328,5.095829,13.733071,0.062008
min,1.000000,1257.000000,19.027422,-166.671575,0.000000
25%,9.630000,26292.000000,34.755302,-96.905129,0.000000
50%,47.290000,48174.000000,39.376593,-87.445204,0.000000
75%,83.010000,72011.000000,41.954163,-80.264637,0.000000
max,22768.110000,99921.000000,66.679297,-66.952026,1.000000


In [55]:
fraud_test_df.head()


,trans_date_trans_time,merchant,amt,zip,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,fraud_Kirlin and Sons,2.86,29209,33.986391,-81.200714,0
1,2020-06-21 12:14:33,fraud_Sporer-Keebler,29.84,84002,39.450498,-109.960431,0
2,2020-06-21 12:14:53,"fraud_Swaniawski, Nitzsche and Welch",41.28,11710,40.495810,-74.196111,0
3,2020-06-21 12:15:15,fraud_Haley Group,60.05,32780,28.812398,-80.883061,0
4,2020-06-21 12:15:17,fraud_Johnston-Casper,3.19,49632,44.959148,-85.884734,0


In [43]:
# Check the fraud_train_df information

fraud_train_df.info()
fraud_train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   merchant               1296675 non-null  object 
 2   amt                    1296675 non-null  float64
 3   zip                    1296675 non-null  int64  
 4   merch_lat              1296675 non-null  float64
 5   merch_long             1296675 non-null  float64
 6   is_fraud               1296675 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 69.3+ MB


,amt,zip,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,7.035104e+01,4.880067e+04,3.853734e+01,-9.022646e+01,5.788652e-03
std,1.603160e+02,2.689322e+04,5.109788e+00,1.377109e+01,7.586269e-02
min,1.000000e+00,1.257000e+03,1.902779e+01,-1.666712e+02,0.000000e+00
25%,9.650000e+00,2.623700e+04,3.473357e+01,-9.689728e+01,0.000000e+00
50%,4.752000e+01,4.817400e+04,3.936568e+01,-8.743839e+01,0.000000e+00
75%,8.314000e+01,7.204200e+04,4.195716e+01,-8.023680e+01,0.000000e+00
max,2.894890e+04,9.978300e+04,6.751027e+01,-6.695090e+01,1.000000e+00


Step 2. Create the labels set (y) from the “is_fraud” column, and then create the features (X) DataFrame from the remaining columns. Note: the data is already split into separate train and test datasets.

In [44]:
# Separate the data into labels and features

# Separate the y variable, the labels
y_train = fraud_train_df['is_fraud']
y_test = fraud_test_df['is_fraud']

# Separate the X variable, the features
X_train = fraud_train_df.drop(columns='is_fraud')
X_test = fraud_test_df.drop(columns='is_fraud')

In [45]:
# Review the y_train variable Series
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [46]:
# Review the y_test variable Series
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

In [47]:
# For X_train Identify the columns that need to be preprocessed
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns


In [48]:
# Pre-process the X_train data.
# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_cols)   # Encode categorical columns
    ])

# Fit and transform the data
X_train = preprocessor.fit_transform(X_train)

In [49]:
# For X_test Identify the columns that need to be preprocessed
categorical_cols = X_test.select_dtypes(include=['object']).columns
numerical_cols = X_test.select_dtypes(include=['int64', 'float64']).columns


In [50]:
# Pre-process the X_test data.
# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_cols)   # Encode categorical columns
    ])

# Fit and transform the data
X_test = preprocessor.fit_transform(X_test)

In [51]:
# Review the X variable DataFrame
X_train

<1296675x1275488 sparse matrix of type '<class 'numpy.float64'>'
	with 7780050 stored elements in Compressed Sparse Row format>

In [52]:
# Review the X variable DataFrame
X_test

<555719x545457 sparse matrix of type '<class 'numpy.float64'>'
	with 3334314 stored elements in Compressed Sparse Row format>

## Create a Logistic Regression Model with the Original Data

Step 1: Fit a logistic regression model by using the training data (X_train and y_train).

In [56]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

Step 2: Save the predictions on the testing data labels by using the testing feature data (X_test) and the fitted model.

In [57]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

predictions_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

predictions_df.head()

ValueError: X has 545457 features, but LogisticRegression is expecting 1275488 features as input.

Step 3: Evaluate the model’s performance by doing the following:
* Generate a confusion matrix.

* Print the classification report.

In [34]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

print("Confusion Matrix")
cm_df

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,553499,75
Actual 1,1769,376


In [35]:
# Print the classification report for the model
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.83      0.18      0.29      2145

    accuracy                           1.00    555719
   macro avg       0.92      0.59      0.64    555719
weighted avg       1.00      1.00      1.00    555719

